In [ ]:
import sys
sys.version

In [ ]:
try:
    from google.colab import output
    !pip install rai==0.2.0-dev5
    
except ModuleNotFoundError:
    %load_ext autoreload
    %autoreload 2

In [ ]:
from io import BytesIO
from zipfile import ZipFile
import tqdm
import numpy as np
import skimage.measure
import matplotlib.pyplot as plt

In [ ]:
import pydicom
pydicom.__version__

In [ ]:
import rai
rai.__version__

In [ ]:
import raicontours

from raicontours import TG263

raicontours.__version__

In [ ]:
cfg = raicontours.get_config()

In [ ]:
cfg

In [ ]:
# TODO: Save training, validation, and hold out uids within raicontours
# Have this downloader be able to receive "training", "validation", or "hold out"
# Have a random one be used per user.
image_paths, structure_path = rai.download_hnscc_example()
structure_path

In [ ]:
sorted_image_series = rai.paths_to_sorted_image_series(paths=image_paths)

In [ ]:
# from rai.legacy.dicom._uid import generate_uid

# common_uid = generate_uid()[0:-10]


# for i, ds in enumerate(sorted_image_series):
#     ds.ImagePositionPatient = np.round(ds.ImagePositionPatient, decimals=4).tolist()
#     ds.SliceLocation = ds.ImagePositionPatient[-1]
#     ds.SliceThickness = np.round(float(ds.SliceThickness), decimals=4)
#     ds.PixelSpacing = np.round(ds.PixelSpacing, decimals=4).tolist()
#     ds.file_meta.ImplementationClassUID = rai.RAI_CONTOURS_IMPLEMENTATION_CLASS_UID
#     ds.file_meta.ImplementationVersionName = rai.RAI_IMPLEMENTATION_VERSION_NAME
#     ds.SOPInstanceUID = f"{common_uid}.{i}"
    
# sorted_image_series[0]

In [ ]:
grids, image_stack = rai.sorted_image_series_to_image_stack_hfs(
    cfg=cfg, sorted_image_series=sorted_image_series)

In [ ]:
models = rai.load_models(cfg=cfg)

In [ ]:
predicted_masks = rai.inference(
    cfg=cfg,
    models=models,
    image_stack=image_stack,
    max_batch_size=10,
    step_size=32,
)

In [ ]:
predicted_contours_by_structure = rai.masks_to_contours_by_structure(
    grids=grids, 
    masks=predicted_masks, 
    structure_names=cfg["structures"],
)

In [ ]:
created_structure_ds = rai.create_dicom_structure_set(
    cfg=cfg, 
    image_series_headers=sorted_image_series, 
    contours_by_structure=predicted_contours_by_structure
)

In [ ]:
# created_structure_ds.remove_private_tags()

In [ ]:
# created_structure_ds.ROIContourSequence

In [ ]:
# sorted_image_series[0]

In [ ]:
structure_filename = "RS-RAi.dcm"

pydicom.dcmwrite(filename=structure_filename, dataset=created_structure_ds, write_like_original=False)

In [ ]:
with ZipFile('results.zip', 'w') as results_zip:
    for path in image_paths:
        results_zip.write(path, path.name)

    results_zip.write(structure_filename)
    

In [ ]:
# Provide the following link to users to view the DICOM files

# https://viewer.imaging.datacommons.cancer.gov/local

In [ ]:
structure_ds = pydicom.read_file(structure_path)
[item.ROIName for item in structure_ds.StructureSetROISequence]

In [ ]:
# image_uids = [ds.SOPInstanceUID for ds in sorted_image_series]


# align_map = {
#     "Brain": [TG263.Brain],
#     "Brainstem": [TG263.Brainstem],
#     "Cochlea-Lt": [TG263.Cochlea_L],
#     "Cochlea-Rt": [TG263.Cochlea_R],
#     "Lacrimal-Lt": [TG263.Glnd_Lacrimal_L],
#     "Lacrimal-Rt": [TG263.Glnd_Lacrimal_R],
#     "Lens-Lt": [TG263.Lens_L],
#     "Lens-Rt": [TG263.Lens_R],
#     "Lung-Lt": [TG263.Lung_L],
#     "Lung-Rt": [TG263.Lung_R],
#     "Mandible": [TG263.Bone_Mandible],
#     "Optic-Nerve-Lt": [TG263.OpticNrv_L],
#     "Optic-Nerve-Rt": [TG263.OpticNrv_R],
#     "Orbit-Lt": [TG263.Eye_L],
#     "Orbit-Rt": [TG263.Eye_R],
#     "Parotid-Lt": [TG263.Parotid_L],
#     "Parotid-Rt": [TG263.Parotid_R],
#     "Spinal-Cord": [TG263.SpinalCord],
#     "Submandibular-Lt": [TG263.Glnd_Submand_L],
#     "Submandibular-Rt": [TG263.Glnd_Submand_R],
# }
# structure_names = list(align_map.keys())

# dicom_contours_by_structure = rai.dicom_to_contours_by_structure(
#     ds=structure_ds, image_uids=image_uids, structure_names=structure_names
# )

In [ ]:
# aligned_predicted_contours_by_structure = rai.merge_contours_by_structure(
#     predicted_contours_by_structure, align_map
# )

# dice = {}
# for name in align_map:
#     dice[name] = rai.dice_from_contours_by_slice(
#         dicom_contours_by_structure[name],
#         aligned_predicted_contours_by_structure[name],
#     )
    
# dice

In [ ]:
# TODO:
# * Calculate and display hausdorff
# * Turn into DICOM file, use online web DICOM viewer to view the file